In [1]:
import pandas as pd

In [2]:
customers = pd.read_csv('customers.csv')
transactions = pd.read_csv('transactions.csv')

In [3]:
customers.head()

,id,cust_name,num_products,province,len_relationship,cust_category
0,1001,Johnny Awesome,3,ON,1,wealth
1,1002,Bob Marley,4,ON,1,personal_banking
2,1004,Taylor Swift,2,AB,2,hnw
3,1101,James Dean,3,BC,3,wealth
4,1006,Stephen Smith,5,ON,4,hnw


In [4]:
transactions.head()

,id,customer_id,txn_type,txn_total
0,1,1001,Debit,165.78
1,2,1001,Credit,42.10
2,4,1001,Credit,103.03
3,5,1001,Credit,56.60
4,6,1002,Debit,214.34


In [5]:
customers = customers.rename(columns={'id':'customer_id'}).set_index('customer_id')

In [6]:
transactions = transactions.set_index('customer_id').rename(columns={'id':'txn_id'})

In [7]:
customers.head()

,cust_name,num_products,province,len_relationship,cust_category
customer_id,,,,,
1001,Johnny Awesome,3,ON,1,wealth
1002,Bob Marley,4,ON,1,personal_banking
1004,Taylor Swift,2,AB,2,hnw
1101,James Dean,3,BC,3,wealth
1006,Stephen Smith,5,ON,4,hnw


In [8]:
transactions.head()

,txn_id,txn_type,txn_total
customer_id,,,
1001,1,Debit,165.78
1001,2,Credit,42.10
1001,4,Credit,103.03
1001,5,Credit,56.60
1002,6,Debit,214.34


Write a function to display a Series object with only the names of Wealth customers.

In [9]:
customers[customers['cust_category'] == 'wealth']\
    .set_index('cust_name')\
        ['cust_category']

cust_name
Johnny Awesome    wealth
James Dean        wealth
Maria Alva        wealth
Steven Prime      wealth
Name: cust_category, dtype: object

Write a function to identify all PersonalBanking customers with more than 3 products.

In [10]:
customers[(customers['cust_category'] == 'personal_banking') & (customers['num_products'] > 3)]

,cust_name,num_products,province,len_relationship,cust_category
customer_id,,,,,
1002,Bob Marley,4,ON,1,personal_banking
1354,Jessica Fast,4,BC,5,personal_banking


What is the average spend of customers in Alberta?

In [11]:
joined = customers.join(transactions)

In [12]:
joined.head()

,cust_name,num_products,province,len_relationship,cust_category,txn_id,txn_type,txn_total
customer_id,,,,,,,,
1001,Johnny Awesome,3,ON,1,wealth,1.0,Debit,165.78
1001,Johnny Awesome,3,ON,1,wealth,2.0,Credit,42.10
1001,Johnny Awesome,3,ON,1,wealth,4.0,Credit,103.03
1001,Johnny Awesome,3,ON,1,wealth,5.0,Credit,56.60
1002,Bob Marley,4,ON,1,personal_banking,6.0,Debit,214.34


In [13]:
joined[joined["province"]=="AB"].groupby("province")["txn_total"].mean()

province
AB    208.976364
Name: txn_total, dtype: float64

How many personal banking customers are there in each province?

In [14]:
joined[joined["cust_category"]=="personal_banking"]\
    [["province","cust_category"]]\
        .rename(columns={"cust_category":"num_of_cust"})\
            .groupby("province")["num_of_cust"]\
                .count()

province
AB    3
BC    6
ON    6
Name: num_of_cust, dtype: int64

Who are the top 3 customers with the highest spend?

In [15]:
joined.head()

,cust_name,num_products,province,len_relationship,cust_category,txn_id,txn_type,txn_total
customer_id,,,,,,,,
1001,Johnny Awesome,3,ON,1,wealth,1.0,Debit,165.78
1001,Johnny Awesome,3,ON,1,wealth,2.0,Credit,42.10
1001,Johnny Awesome,3,ON,1,wealth,4.0,Credit,103.03
1001,Johnny Awesome,3,ON,1,wealth,5.0,Credit,56.60
1002,Bob Marley,4,ON,1,personal_banking,6.0,Debit,214.34


In [16]:
joined.groupby("customer_id").agg({"txn_total": "sum"}).sort_values("txn_total", ascending=False).head(3)

,txn_total
customer_id,
1354,909.87
1585,858.55
1285,811.40


Which customer has the highest single transaction?

In [17]:
joined.sort_values("txn_total", ascending=False).head(1)

,cust_name,num_products,province,len_relationship,cust_category,txn_id,txn_type,txn_total
customer_id,,,,,,,,
1014,Delilah Avery,2,AB,3,personal_banking,20.0,Debit,449.39


What is the average spend by Province and Customer Category?

In [18]:
joined.groupby(["province","cust_category"]).agg({"txn_total": "mean"}).unstack("province")

txn_total                            
province                  AB      BC      MB          ON
cust_category                                           
hnw               147.952000     NaN  199.73  225.890000
personal_banking  233.476667  285.26     NaN  172.300000
wealth            286.183333     NaN     NaN  168.415714

What is the average spend by transaction type (e.g. credit, debit, cash)?

In [19]:
transactions.head()

,txn_id,txn_type,txn_total
customer_id,,,
1001,1,Debit,165.78
1001,2,Credit,42.10
1001,4,Credit,103.03
1001,5,Credit,56.60
1002,6,Debit,214.34


In [20]:
transactions.groupby("txn_type").agg({"txn_total": "mean"})

,txn_total
txn_type,
Cash,182.262727
Credit,206.926667
Debit,227.204286


Which transaction type is the most popular in each province?

In [21]:
joined.groupby(["province", "txn_type"]).agg({"txn_type":"count"}).unstack()

txn_type             
txn_type     Cash Credit Debit
province                      
AB            5.0    2.0   4.0
BC            NaN    2.0   4.0
MB            2.0    1.0   NaN
ON            4.0    4.0   6.0

In [22]:
joined.groupby(["province", "txn_type"]).agg({"txn_type":"count"}).unstack().info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, AB to ON
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   (txn_type, Cash)    3 non-null      float64
 1   (txn_type, Credit)  4 non-null      float64
 2   (txn_type, Debit)   3 non-null      float64
dtypes: float64(3)
memory usage: 128.0+ bytes
